In [5]:
#!pip install scikit-learn==1.3.0 tensorflow==2.13.0


In [6]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import GridSearchCV

import numpy as np
from tensorflow.keras.datasets import cifar100
import matplotlib.pyplot as plt


In [7]:
(train_images,train_labels),(test_images,test_labels)=cifar100.load_data()

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [8]:
train_images.ndim

4

In [9]:
train_images.shape

(50000, 32, 32, 3)

In [10]:
#  train_images    # to check the train_images they are not scaled

In [11]:
# scale the train images
train_images,test_images=(train_images-127.5)/127.5,(test_images-127.5)/127.5

train_labels=to_categorical(train_labels,100)
test_labels=to_categorical(test_labels,100)

In [12]:
# train_images    # now image scaled between [-1,1]

In [13]:
#  train_labels   # coverte into class labels

# **DEFINE THE LENET-5 MODEL**

In [14]:
from tensorflow.keras.layers import Conv2D,AveragePooling2D,Flatten,Dense,Dropout
from tensorflow.keras.models import Sequential

In [20]:
def LeNet_5(learning_rate=0.001):
  model=Sequential()

  model.add(Conv2D(6,kernel_size=(5,5),strides=(1,1),activation='tanh',input_shape=(32,32,3)))
  model.add(AveragePooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(16,kernel_size=(5,5),strides=(1,1),activation='tanh'))
  model.add(AveragePooling2D(pool_size=(2,2),strides=(2,2)))

  # fully connected convolutional layer
  model.add(Conv2D(120,kernel_size=(5,5), activation='tanh'))

  # flattening the images'
  model.add(Flatten())

  # fully connected layer with 84 neurons
  model.add(Dense(84, activation='relu'))

  # output layer with 100 meurons for each class
  model.add(Dense(100, activation='softmax'))

  # Compile the model
  optimizer = Adam(learning_rate=learning_rate)
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

  return model


model=LeNet_5()

In [21]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 28, 28, 6)           │             456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_4                  │ (None, 14, 14, 6)           │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 10, 10, 16)          │           2,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_5                  │ (None, 5, 5, 16)            │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 1, 1, 120)           │          48,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 84)                  │          10,164 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 100)                 │           8,500 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 69,656 (272.09 KB)

 Trainable params: 69,656 (272.09 KB)

 Non-trainable params: 0 (0.00 B)

# Train the model

In [22]:
# using the earlystopping and learning rate reduction for better training

early_stopping=EarlyStopping(monitor='val_loss',patience=5)
lr_reduction=ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=2,min_lr=0.00001)


# train the model

train=model.fit(train_images,train_labels,epochs=60,validation_split=0.2,batch_size=128,callbacks=[early_stopping,lr_reduction])

Epoch 1/60
313/313 ━━━━━━━━━━━━━━━━━━━━ 31s 92ms/step - accuracy: 0.0622 - loss: 4.2224 - val_accuracy: 0.1311 - val_loss: 3.7370 - learning_rate: 0.0010
Epoch 2/60
313/313 ━━━━━━━━━━━━━━━━━━━━ 41s 94ms/step - accuracy: 0.1508 - loss: 3.6296 - val_accuracy: 0.1689 - val_loss: 3.5260 - learning_rate: 0.0010
Epoch 3/60
313/313 ━━━━━━━━━━━━━━━━━━━━ 40s 92ms/step - accuracy: 0.1917 - loss: 3.4042 - val_accuracy: 0.1906 - val_loss: 3.4047 - learning_rate: 0.0010
Epoch 4/60
313/313 ━━━━━━━━━━━━━━━━━━━━ 41s 92ms/step - accuracy: 0.2145 - loss: 3.2786 - val_accuracy: 0.2077 - val_loss: 3.3292 - learning_rate: 0.0010
Epoch 5/60
313/313 ━━━━━━━━━━━━━━━━━━━━ 41s 93ms/step - accuracy: 0.2293 - loss: 3.1766 - val_accuracy: 0.2125 - val_loss: 3.2835 - learning_rate: 0.0010
Epoch 6/60
313/313 ━━━━━━━━━━━━━━━━━━━━ 29s 92ms/step - accuracy: 0.2432 - loss: 3.1094 - val_accuracy: 0.2208 - val_loss: 3.2657 - learning_rate: 0.0010
Epoch 7/60
313/313 ━━━━━━━━━━━━━━━━━━━━ 43s 98ms/step - accuracy: 0.2594 - l

In [23]:
# Evaludate the model

test_loss,test_acc=model.evaluate(test_images,test_labels)
print(f"test_acc {test_acc}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.2567 - loss: 3.1272
test_acc 0.2556000053882599
